# Using Hugging Face Inference with KDB.AI to Create a AI Tool Search Engine

##### Note: This example requires a KDB.AI endpoint and API key. Sign up for a free [KDB.AI account](https://kdb.ai/get-started).

How to get started with using the Huggingface Inference API with KDB.AI.

You will learn how to:

1. Connect to KDB.AI
2. Create a KDB.AI Database & Table
3. Load Data
4. Use the Sentence Transformers library to embed every description in the dataset
5. Insert the data into our KDB.AI table
6. Perform Similarity Search using the Huggingface Inference API
7. Delete the KDB.AI Database & Table to Conserve Resources

# Why Use Hugging Face for Embeddings?

When building production applications that utilize embeddings, it's often advantageous to use open-source embedding models for several reasons:

1. **Control**: Open-source models give developers more control over the embeddings process, reducing dependence on third-party embedding providers.

2. **Local Embedding**: With open-source models, you can create embeddings locally, which is particularly useful for embedding your dataset.

A common approach is to use a Python framework like sentence-transformers, developed by Hugging Face, which offers state-of-the-art sentence, text, and image embeddings. Here's a typical workflow:

1. **Embed your dataset locally**: Use a library like Sentence Transformers to embed your dataset, which might consist of AI tools and associated metadata.

2. **Embed queries at inference time**: When a user submits a query, use an external service like Hugging Face's Inference API to embed the query. This eliminates the need to deploy your own model, allowing you to leverage a fully optimized external service.

By following this approach, you can build a system that searches through hundreds of AI tools without the need to deploy any infrastructure (and scale to millions!). Additionally, since you embed the dataset locally, you can use Hugging Face's free plan without requiring a credit card or worrying about hitting rate limits, at least until you are ready for production.

In this tutorial, we will walk through the process of embedding a dataset of AI tools using Sentence Transformers, and then using Hugging Face's Inference API to embed queries at inference time, enabling efficient and scalable search capabilities.

You will need a Hugging Face api token for this sample. Please create a Hugging Face account by going to [Hugging Face – The AI community building the future](https://huggingface.co/) and create a token by going to https://huggingface.co/settings/tokens

You can then enter this token below or set it to HF_TOKEN in your environment.

# 0. Setup

### Install dependencies

In order to successfully run this sample, note the following steps depending on where you are running this notebook:

-***Run Locally / Private Environment:*** The [Setup](https://github.com/KxSystems/kdbai-samples/blob/main/README.md#setup) steps in the repository's `README.md` will guide you on prerequisites and how to run this with Jupyter.


-***Colab / Hosted Environment:*** Open this notebook in Colab and run through the cells.

In [ ]:
!pip install kdbai_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 6.2 MB/s eta 0:00:00


In [ ]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

### Import Packages

In [ ]:
# vector DB
import os
from getpass import getpass
import kdbai_client as kdbai
import time

In [ ]:
import numpy as np
import pandas as pd

# 1. Connect to KDB.AI

### Define KDB.AI Session

KDB.AI comes in two offerings:

1. [KDB.AI Cloud](https://trykdb.kx.com/kdbai/signup/) - For experimenting with smaller generative AI projects with a vector database in our cloud.
2. [KDB.AI Server](https://trykdb.kx.com/kdbaiserver/signup/) - For evaluating large scale generative AI applications on-premises or on your own cloud provider.

Depending on which you use there will be different setup steps and connection details required.

##### Option 1. KDB.AI Cloud

To use KDB.AI Cloud, you will need two session details - a URL endpoint and an API key.
To get these you can sign up for free [here](https://trykdb.kx.com/kdbai/signup).

You can connect to a KDB.AI Cloud session using `kdbai.Session` and passing the session URL endpoint and API key details from your KDB.AI Cloud portal.

If the environment variables `KDBAI_ENDPOINTS` and `KDBAI_API_KEY` exist on your system containing your KDB.AI Cloud portal details, these variables will automatically be used to connect.
If these do not exist, it will prompt you to enter your KDB.AI Cloud portal session URL endpoint and API key details.

In [ ]:
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else getpass("KDB.AI API key: ")
)

KDB.AI endpoint: https://cloud.kdb.ai/instance/bbdfv0a29f
KDB.AI API key: ··········


In [ ]:
HF_TOKEN = (
    os.environ["HF_TOKEN"]
    if "HF_TOKEN" in os.environ
    else getpass("Hugging Face token: ")
)

Hugging Face token: ··········


In [ ]:
session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

##### Option 2. KDB.AI Server

To use KDB.AI Server, you will need download and run your own container.
To do this, you will first need to sign up for free [here](https://trykdb.kx.com/kdbaiserver/signup/).

You will receive an email with the required license file and bearer token needed to download your instance.
Follow instructions in the signup email to get your session up and running.

Once the [setup steps](https://code.kx.com/kdbai/gettingStarted/kdb-ai-server-setup.html) are complete you can then connect to your KDB.AI Server session using `kdbai.Session` and passing your local endpoint.

In [ ]:
# session = kdbai.Session(endpoint="http://localhost:8082")

### Verify Defined Databases

We can check our connection using the `session.databases()` function.
This will return a list of all the databases we have defined in our vector database thus far.
This should return a "default" database along with any other databases you have already created.

In [ ]:
session.databases()

[KDBAI database "chunking_comparison",
 KDBAI database "default",
 KDBAI database "documents",
 KDBAI database "search_eval_db"]

### Create a Database Called "myDatabase"

In [ ]:
# ensure no database called "myDatabase" exists
try:
    session.database("myDatabase").drop()
except kdbai.KDBAIException:
    pass

In [ ]:
# Create the database
db = session.create_database("myDatabase")

# 2. Create a KDB.AI Table

To create a table we can use `create_table`, this function takes two arguments - the name and schema of the table.

This schema must meet the following criteria:
- It must contain a list of columns.
- All columns must have either a `type` or a `qtype`.
- One column of vector embeddings, this column is implicitly an array of `float64s`.

### Define Schema
The schema contains all metadata columns, and a 'description_embedding' column which will be used for similarity search


In [ ]:
schema = [
        {"name": "id", "type": "str"},
        {"name": "name", "type": "str"},
        {"name": "description", "type": "str"},
        {"name": "summary", "type": "str"},
        {"name": "title", "type": "str"},
        {"name": "visitors", "type": "int64"},
        {"name": "description_embedding", "type": "float64s"},
    ]

### Define the indexes
We will define our dimensionality, similarity metric and index type with the vectorIndex attribute. For this example we chose:

- type = hnsw : HNSW enhances efficiency while maintaining accuracy. You have the choice of using other indexes like, qHNSW, and IVFPQ, qFlat or a Flat index here, as with metrics the one you chose depends your data and your overall performance requirements.
- name = hnsw_index : this is a custom name you give your index.

#### params:
- dims = 384 : In the next section, we generate embeddings that are 384-dimensional to match this. The number of dimensions should mirror the output dimensions of your embedding model.
- metric = L2 : We chose L2/Euclidean distance. Our dummy dataset is low dimensional which Euclidean distance is suitable for. You have the choice of using other metrics here like IP/Inner Product and CS/Cosine Similarity and the one you chose depends on the specific context and nature of your data.

!Note, it is possible to define multiple indexes within a table!

In [ ]:
# Define the index
indexes = [
    {
        'type': 'hnsw',
        'name': 'hnsw_index',
        'column': 'description_embedding',
        'params': {'dims': 384, 'metric': "L2"},
    },
]


### Create Table

In [ ]:
table = db.create_table(table="ai_tools", schema=schema, indexes=indexes)

# 3. Load Data

We fetch data from a github gist containing companies, descriptions, and some metadata. We will then add these to pandas dataframe with column names/types matching the target table.

In [ ]:
import requests

gist_url = "https://gist.github.com/mrmps/2f62a2287cb2c1ca63a2762fcaac89bc/raw"
response = requests.get(gist_url)
ai_tools_data = response.json()
df = pd.DataFrame.from_dict(ai_tools_data)

# drop column with unecessary metadata
df.drop(columns=["xata"], inplace=True)
df.head()

,description,id,name,summary,title,visitors
0,Generate 3D textures for your game in seconds ...,rec_cfn1112cibvc11jnn2qg,TextureLab,TextureLab is a website that provides 3D textu...,Instant And Unique 3D Textures For Your Next G...,23913
1,Luma Labs enables users to explore 3D modeling...,rec_cfn1112cibvc11jnn2r0,lumalabs,Luma Labs is a website that offers an early ex...,Imagine 3D V1.2 (Alpha),456963
2,Make motion capture from video easier and more...,rec_cfn1112cibvc11jnn2rg,plask,Plask is an AI-powered mocap animation tool th...,Ai-Powered Mocap Animation Tool.,90960
3,Get hundreds of interior design ideas for your...,rec_cfn1112cibvc11jnn2s0,AI Room Planner,AI Room Planner is an online platform that uti...,Interior Design By Ai,211540
4,A platform powered by AI to help you create be...,rec_cfn1112cibvc11jnn2sg,AI TWO,AI TWO is a website that provides a platform f...,Aitwo.Co - The Ai-Powered All-In-One Design Pl...,7201


# 4. Use the Sentence Transformers library to embed every description in the dataset

We set the embedding model to BAAI/bge-small-en-v1.5, which is a fast and small model. This is what we will use during inference time as well.

If you want faster inference, you can try the [FastEmbed](https://github.com/qdrant/fastembed) library, a much faster and more lightweight embedding library.

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-small-en-v1.5")

descriptions = [tool["description"] for tool in ai_tools_data]
embeddings = model.encode(descriptions)

In [ ]:
embeddings

array([[-0.06802838,  0.01769781,  0.07132471, ...,  0.04166842,
        -0.01963803, -0.03634397],
       [ 0.00284368,  0.00349111,  0.03926528, ..., -0.01490239,
         0.00412081,  0.02246647],
       [-0.0853649 , -0.05372242,  0.01503714, ...,  0.01607882,
         0.04058059, -0.02476995],
       ...,
       [ 0.00551534, -0.02548733, -0.00431464, ..., -0.00406336,
         0.06047559, -0.03689234],
       [-0.08149452, -0.00607408, -0.00040344, ...,  0.02765153,
         0.04479543, -0.00464933],
       [-0.09128135, -0.05604198,  0.01856983, ...,  0.01355307,
         0.05817634, -0.05754768]], dtype=float32)

# 5. Insert the data into our KDB.AI table

In [ ]:
# Create a DataFrame with the AI tools data
data = pd.DataFrame(ai_tools_data)[["id", "name", "description", "summary", "title", "visitors"]]
data["description_embedding"] = embeddings.tolist()

# Bulk insert the data into KDB.AI
table.insert(data)

{'rowsInserted': 851}

## Confirm data is loaded correctly

In [ ]:
table.query()

,id,name,description,summary,title,visitors,description_embedding
0,rec_cfn1112cibvc11jnn2qg,TextureLab,Generate 3D textures for your game in seconds ...,TextureLab is a website that provides 3D textu...,Instant And Unique 3D Textures For Your Next G...,23913,"[-0.06802837550640106, 0.017697805538773537, 0..."
1,rec_cfn1112cibvc11jnn2r0,lumalabs,Luma Labs enables users to explore 3D modeling...,Luma Labs is a website that offers an early ex...,Imagine 3D V1.2 (Alpha),456963,"[0.002843681490048766, 0.003491112031042576, 0..."
2,rec_cfn1112cibvc11jnn2rg,plask,Make motion capture from video easier and more...,Plask is an AI-powered mocap animation tool th...,Ai-Powered Mocap Animation Tool.,90960,"[-0.08536490052938461, -0.05372241884469986, 0..."
3,rec_cfn1112cibvc11jnn2s0,AI Room Planner,Get hundreds of interior design ideas for your...,AI Room Planner is an online platform that uti...,Interior Design By Ai,211540,"[0.02065597102046013, 0.02826962247490883, 0.0..."
4,rec_cfn1112cibvc11jnn2sg,AI TWO,A platform powered by AI to help you create be...,AI TWO is a website that provides a platform f...,Aitwo.Co - The Ai-Powered All-In-One Design Pl...,7201,"[-0.022134779021143913, -0.03189411759376526, ..."
...,...,...,...,...,...,...,...
846,rec_cod2au57l1i4603r3hvg,Scott Krager,Thumbnails.com uses AI to generate dozens of u...,Unlock the power of eye-catching thumbnails wi...,Thumbnails.com,0,"[-0.07755478471517563, -0.059786420315504074, ..."
847,rec_codntepuqmnhe7ku1ing,Nen Fard,"StockTune: AI-powered, public-domain music for...",\nStockTune is a revolutionary platform offeri...,StockTune,0,"[-0.03542690724134445, -0.05728309601545334, -..."
848,rec_codr709uqmnhe7ku1te0,Nen Fard,"StockCake: Free, AI-generated stock photos in...",StockCake is a revolutionary stock photo site ...,StockCake,0,"[0.005515342578291893, -0.025487331673502922, ..."
849,rec_coidgc9uqmnhe7l0eug0,Jason West,FastBots enables anyone to quickly create a po...,FastBots is a no-code AI chatbot builder for b...,FastBots,0,"[-0.0814945176243782, -0.006074081175029278, -..."


# 6. Perform Similarity Search Using the Hugging Face Inference API

## Embed the Query with the Hugging Face Inference API
Use the Hugging Face Inference API to embed the query so that it can be used to search our index

!! Note that you might need to run this cell a few times as it takes a few seconds for the model to be ready.

In [ ]:
# Perform a search using Hugging Face embeddings
import requests

# make sure your URL looks like this to ensure you get instant results, and not a model loading error
embedding_url = "https://api-inference.huggingface.co/pipeline/feature-extraction/BAAI/bge-small-en-v1.5"

def generate_query_embedding(text: str) -> list[float]:
    response = requests.post(
        embedding_url,
        headers={"Authorization": f"Bearer {HF_TOKEN}", "x-wait-for-model": "true"},
        json={"inputs": text}
    )

    if response.status_code != 200:
        raise ValueError(f"Request failed with status code {response.status_code}: {response.text}")
    return response.json()

query = "AI tool for creating 3D textures"
query_embedding = generate_query_embedding(query)

## Run the query with our query embedding

We are searching based on the description for the most relevant startups to the query. Remember that "hnsw_index" is the index name we created when defining our index before creating the table.

In [ ]:
results = table.search(vectors={"hnsw_index":[query_embedding]},n=3,)

results[0]

,__nn_distance,id,name,description,summary,title,visitors,description_embedding
0,0.25221,rec_cfn1112cibvc11jnn2qg,TextureLab,Generate 3D textures for your game in seconds ...,TextureLab is a website that provides 3D textu...,Instant And Unique 3D Textures For Your Next G...,23913,"[-0.06802837550640106, 0.017697805538773537, 0..."
1,0.26723,rec_cfn11a2cibvc11jnndbg,Ponzu.gg,Create realistic 3D images with AI-generated t...,Ponzu is a website that helps 3D artists and d...,Ponzu.,6526,"[-0.06463479995727539, -0.014672139659523964, ..."
2,0.34271,rec_cfn119acibvc11jnncf0,Masterpiece Studio,Create 3D models with Generative AI and deploy...,Masterpiece Studio is a company that has devel...,Masterpiece Studio.,38954,"[-0.041312605142593384, -0.003570165950804949,..."


# 7. Delete the KDB.AI Table & Database to Conserve Resources


We can use `table.drop()` to delete a table, and db.drop() to delete the database.

In [ ]:
table.drop()
db.drop()

True

<div class="alert alert-block alert-warning">
<b>Warning:</b> Once you drop a table, you cannot use it again.
</div>

## Take Our Survey

We hope you found this sample helpful! Your feedback is important to us, and we would appreciate it if you could take a moment to fill out our brief survey. Your input helps us improve our content.

[**Take the Survey**](https://delighted.com/t/UGvwprmK)

## Next Steps

Now that you’re successfully making indexes with KDB.AI, you can start inserting your own data or view more examples:
- [PDF Document Search](../document_search)
- [MRI Image Search](../image_search)
- [Music Recommendation System](../music_recommendation)
- [Sensor Pattern Matching](../pattern_matching)
- [Retrieval Augmented Generation with LangChain](../retrieval_augmented_generation)
- [Sentiment Analysis of Reviews](../sentiment_analysis)